In [1]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import numpy as np
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import scipy
import xarray as xr
from tqdm import tqdm
import glob
# from devtools import pprint
from rich import print
from tqdm import tqdm

In [2]:
# general eWC
import ewatercycle
import ewatercycle.forcing
import ewatercycle.models

In [3]:
# pip install --upgrade ewatercycle

In [4]:
path = Path.cwd()
forcing_path = path / "Forcing"
observations_path = path / "Observations"
figure_path = path / "Figures"
output_path = path / "Output"
forcing_path

PosixPath('/home/dhaasnoot/DA/Forcing')

Simple example using HBV:

In [5]:
from ewatercycle.forcing import sources

In [6]:
# pip uninstall ewatercycle_DA -y

In [7]:
# pip install ewatercycle_DA==0.0.5

In [8]:
from ewatercycle_DA import DA

In [9]:
import ewatercycle_DA

In [10]:
ewatercycle_DA.__version__

'0.0.5'

In [11]:
n_particles = 10

In [12]:
ensemble = DA.Ensemble(N=n_particles)
ensemble.setup()

In [13]:
experiment_start_date = "1997-08-01T00:00:00Z"
experiment_end_date = "1999-03-01T00:00:00Z"
HRU_id = 14138900

In [14]:
forcing_args = []
for i in range(1,11):
    forcing_args.append(dict(
        start_time = experiment_start_date,
        end_time = experiment_end_date,
        directory = forcing_path,
        camels_file = f'0{HRU_id}_lump_cida_forcing_leap.txt', 
        alpha = 1.23 + (i/100)
    ))

In [15]:
%%time
lst_forcing = ensemble.generate_forcing([ewatercycle.forcing.sources.HBVForcing]*10, forcing_args)

CPU times: user 0 ns, sys: 5.77 ms, total: 5.77 ms
Wall time: 5.68 ms


In [16]:
lst_forcing

[HBVForcing(start_time='1997-08-01T00:00:00Z', end_time='1999-03-01T00:00:00Z', directory=PosixPath('/home/dhaasnoot/DA/Forcing'), shape=None, filenames={}, camels_file='014138900_lump_cida_forcing_leap.txt', pr='.nc', pev='.nc', alpha=1.24, test_data_bool=False),
 HBVForcing(start_time='1997-08-01T00:00:00Z', end_time='1999-03-01T00:00:00Z', directory=PosixPath('/home/dhaasnoot/DA/Forcing'), shape=None, filenames={}, camels_file='014138900_lump_cida_forcing_leap.txt', pr='.nc', pev='.nc', alpha=1.25, test_data_bool=False),
 HBVForcing(start_time='1997-08-01T00:00:00Z', end_time='1999-03-01T00:00:00Z', directory=PosixPath('/home/dhaasnoot/DA/Forcing'), shape=None, filenames={}, camels_file='014138900_lump_cida_forcing_leap.txt', pr='.nc', pev='.nc', alpha=1.26, test_data_bool=False),
 HBVForcing(start_time='1997-08-01T00:00:00Z', end_time='1999-03-01T00:00:00Z', directory=PosixPath('/home/dhaasnoot/DA/Forcing'), shape=None, filenames={}, camels_file='014138900_lump_cida_forcing_leap.tx

In [17]:
%%time
ewatercycle.forcing.sources.HBVForcing(**forcing_args[0])

CPU times: user 0 ns, sys: 755 µs, total: 755 µs
Wall time: 636 µs


HBVForcing(start_time='1997-08-01T00:00:00Z', end_time='1999-03-01T00:00:00Z', directory=PosixPath('/home/dhaasnoot/DA/Forcing'), shape=None, filenames={}, camels_file='014138900_lump_cida_forcing_leap.txt', pr='.nc', pev='.nc', alpha=1.24, test_data_bool=False)

## Using ERA5 data

In [18]:
#shapefile that describes the basin we want to study.
path = Path.cwd()
forcing_path = path / "Forcing"
shapeFile = forcing_path/ "Rhine_shp" / "Rhine.shp"

#location to saved forcing results from previous notebook
forcingLocation = forcing_path / "RhineForcing2000-2002"

#GRDC station ID for the observation station
grdc_station_id = "6335020"  # GRDC station ID
basin_name = "Rhine"
observation_path = path / "Observations"

#period of interest. Make sure that GRDC data is available for this period and that.
experiment_start_date="2000-01-01T00:00:00Z"

In [19]:
ewatercycle.forcing.sources["LumpedMakkinkForcing"]

ewatercycle._forcings.makkink.LumpedMakkinkForcing

In [20]:
n_particles = 6

In [21]:
import psutil

In [22]:
# doesn't like the default config so use threadpool, see https://docs.dask.org/en/stable/scheduler-overview.html

In [23]:
from concurrent.futures import ThreadPoolExecutor
# dask_config: dict = {"scheduler": "single-threaded"}
dask_config: dict = {"multiprocessing.context": "forkserver"}

In [24]:
dask_config

{'multiprocessing.context': 'forkserver'}

In [25]:
ensemble = DA.Ensemble(N=n_particles,dask_config=dask_config)
ensemble.setup()

Slightly vary the generate config slightly: <br>
(highly hypothetical yes)

In [26]:
forcing_args = []
for i in range(1,n_particles+1):
    forcing_args.append(dict(
        dataset="ERA5",
        start_time = experiment_start_date,
        end_time = f"2000-0{i}-01T00:00:00Z",
        shape = shapeFile.absolute()
    ))

Wall time: 35.6 s

In [27]:
# %%time
ewatercycle.forcing.sources["LumpedMakkinkForcing"].generate(**forcing_args[0])

LumpedMakkinkForcing(start_time='2000-01-01T00:00:00Z', end_time='2000-01-01T00:00:00Z', directory=PosixPath('/home/dhaasnoot/DA/esmvaltool_output/ewcrepqqtg52p7_20240422_160603/work/diagnostic/script'), shape=PosixPath('/home/dhaasnoot/DA/Forcing/Rhine_shp/Rhine.shp'), filenames={'pr': 'OBS6_ERA5_reanaly_1_day_pr_2000-2000.nc', 'tas': 'OBS6_ERA5_reanaly_1_day_tas_2000-2000.nc', 'rsds': 'OBS6_ERA5_reanaly_1_day_rsds_2000-2000.nc', 'evspsblpot': 'Derived_Makkink_evspsblpot.nc'})

single threaded: Wall time: 2min 37s <br> 
multiprocessing: Wall time: 1min 48s

In [37]:
psutil.cpu_count()

2

In [28]:
%%time
lst_forcing = ensemble.generate_forcing(ewatercycle.forcing.sources["LumpedMakkinkForcing"].generate, forcing_args)

CPU times: user 895 ms, sys: 207 ms, total: 1.1 s
Wall time: 1min 48s


In [29]:
lst_forcing

[LumpedMakkinkForcing(start_time='2000-01-01T00:00:00Z', end_time='2000-01-01T00:00:00Z', directory=PosixPath('/home/dhaasnoot/DA/esmvaltool_output/ewcrep58eqrzvm_20240422_160631/work/diagnostic/script'), shape=PosixPath('/home/dhaasnoot/DA/Forcing/Rhine_shp/Rhine.shp'), filenames={'pr': 'OBS6_ERA5_reanaly_1_day_pr_2000-2000.nc', 'tas': 'OBS6_ERA5_reanaly_1_day_tas_2000-2000.nc', 'rsds': 'OBS6_ERA5_reanaly_1_day_rsds_2000-2000.nc', 'evspsblpot': 'Derived_Makkink_evspsblpot.nc'}),
 LumpedMakkinkForcing(start_time='2000-01-01T00:00:00Z', end_time='2000-02-01T00:00:00Z', directory=PosixPath('/home/dhaasnoot/DA/esmvaltool_output/ewcrep40ssohis_20240422_160705/work/diagnostic/script'), shape=PosixPath('/home/dhaasnoot/DA/Forcing/Rhine_shp/Rhine.shp'), filenames={'pr': 'OBS6_ERA5_reanaly_1_day_pr_2000-2000.nc', 'tas': 'OBS6_ERA5_reanaly_1_day_tas_2000-2000.nc', 'rsds': 'OBS6_ERA5_reanaly_1_day_rsds_2000-2000.nc', 'evspsblpot': 'Derived_Makkink_evspsblpot.nc'}),
 LumpedMakkinkForcing(start_ti